# Check for english in tagged as non-english

## Step 1: Prepare non-english only (not verified)

In [ ]:
import math
import re

import pandas as pd

from accessory import clean_text_data, filter_text_data, get_text_data, word_counter
from comprehend import get_dominant_language

pd.options.mode.chained_assignment = None

In [ ]:
df = pd.read_pickle('available_20200407.pkl')

### Add language column

In [ ]:
def language(hash):
    path = 'data/{}.json'.format(hash)
    return pd.read_json(path).lang

In [ ]:
df['language'] = df.hash.apply(language)

### Filter by language

In [ ]:
dff = df[df.language != 'en']

### Save to pickle file

In [ ]:
dff.to_pickle('non-english_not_verified_20201116.pkl')

## Step 2: Get text

In [ ]:
df = pd.read_pickle('non-english_not_verified_20201116.pkl')

### Add text column

In [ ]:
def get_text(hash):
    path = 'data/{}.json'.format(hash)
    dff = pd.read_json(path, encoding='utf-8')
    children = dff._children[0]
    data = get_text_data(children)
    data = filter_text_data(data)
    data = clean_text_data(data)
    text = ' '.join(data)
    return text

In [ ]:
df['text'] = df.hash.apply(get_text)

### Save to pickle file

In [ ]:
df.to_pickle('non-english_with_text_not_verified_20201116.pkl')

## Step 3: Verify language

In [ ]:
df = pd.read_pickle('non-english_with_text_not_verified_20201116.pkl')

### Add `characters_num` column

In [ ]:
def count_characters(text):
    return len(text)

In [ ]:
df['characters_num'] = df.text.apply(count_characters)

### Add `units_num` column

In [ ]:
def count_units(characters):
    units = characters / 100.0
    units = math.ceil(units)
    if units < 3:
        units = 3
    return units

In [ ]:
df['units_num'] = df.characters_num.apply(count_units)

### Count price of Amazon Comprehend language detection

In [ ]:
df.units_num.sum() * 0.0001

### Add `comprehend` column

In [ ]:
def get_comprehend(text):
    result = get_dominant_language(text)
    if result == 1:
        return None, None
    return result

In [ ]:
df['comprehend'] = df.text.apply(get_comprehend)

### Save to pickle file

In [ ]:
df.to_pickle('non-english_comprehend_20201116.pkl')

## Step 4: Check for english

In [ ]:
df = pd.read_pickle('non-english_comprehend_20201116.pkl')

### Add `language_by_aws` column

In [ ]:
def get_language_by_aws(data):
    return data[0]

In [ ]:
df['language_by_aws'] = df.comprehend.apply(get_language_by_aws)

### Add `aws_score` column

In [ ]:
def get_aws_score(data):
    return data[1]

In [ ]:
df['aws_score'] = df.comprehend.apply(get_aws_score)

### Select only verified english

In [ ]:
dff = df[df.language_by_aws == 'en'][df.aws_score > 0.9]

### Save to pickle file

In [ ]:
dff.to_pickle('english_verified_additional_20201116.pkl')